In [3]:
from tqdm import tqdm
possible_laminations = []
triplets_only = []
for i in range(13):
    possible_laminations.append((i + 1,))

compatibility = {1 : [4, 5, 8, 9, 10],
                 2 : [3, 6, 8, 9, 12],
                 3 : [4, 7, 9, 12],
                 4 : [7, 9, 10],
                 5 : [8, 10, 13],
                 6 : [8, 12, 13],
                 7 : [10, 11, 12],
                 8 : [9, 13],
                 9 : [],
                 10 : [11, 13],
                 11 : [12, 13],
                 12 : [13]}

for i in range(0, 13):
    for j in range(i + 1, 13):
        if (j + 1 in compatibility[i + 1]):
            possible_laminations.append((i + 1, j + 1))

triples = []
counter = 0
for i in range(13):
    for j in range(i + 1, 13):
        for k in range(j + 1, 13):
            if j + 1 in compatibility[i + 1]:
                if k + 1 in compatibility[j + 1]:
                    if k + 1 in compatibility[i + 1]:
                        counter += 1
                        print(counter, end = '')
                        print(')', i + 1, j + 1, k + 1)
                        possible_laminations.append((i + 1, j + 1, k + 1))
                        triplets_only.append([i + 1, j + 1, k + 1])
from itertools import permutations, combinations, combinations_with_replacement
import itertools
from sympy import *
a, b, c, d, q, f, g, h, k = symbols('a b c d q f g h k')
all_triples = []
for ever in itertools.combinations(possible_laminations, 3):
    all_triples.append(tuple(ever))

intersection_index = Matrix([[-1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 1], 
        [1, -1, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 1], 
        [1, 0, -1, 0, 2, 1, 0, 1, 0, 1, 1, 0, 1], 
        [0, 1, 0, -1, 1, 2, 0, 1, 0, 0, 1, 1, 1], 
        [0, 2, 2, 1, -1, 2, 2, 0, 1, 0, 2, 2, 0], 
        [2, 0, 1, 2, 2, -1, 2, 0, 1, 2, 2, 0, 0],
        [1, 1, 0, 0, 2, 2, -1, 2, 1, 0, 0, 0, 1],
        [0, 0, 1, 1, 0, 0, 2, -1, 0, 1, 2, 1, 0],
        [0, 0, 0, 0, 1, 1, 1, 0, -1, 1, 2, 1, 1],
        [0, 2, 1, 0, 0, 2, 0, 1, 1, -1, 0, 1, 0],
        [2, 2, 1, 1, 2, 2, 0, 2, 2, 0, -1, 0, 0],
        [2, 0, 0, 1, 2, 0, 0, 1, 1, 1, 0, -1, 0],
        [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]])
def create_equation_by_matrix(list1, list2, id1, id2):
    eq = 0
    letters1 = [a, b, c]
    letters2 = [d, q, f]
    letters3 = [g, h, k]
    all_letters = [letters1, letters2, letters3]
    A_raw = [0] * 13
    B_raw = [0] * 13
    for i in range(len(list1)):
        A_raw[list1[i] - 1] = all_letters[id1][i]
    for j in range(len(list2)):
        B_raw[list2[j] - 1] = all_letters[id2][j]
    A = Matrix(A_raw)
    B = Matrix(B_raw)
    eq = A.T * intersection_index * B
    return eq[0]
def balanced(eq):
    if eq == 0:
        return True
    pos = 0
    neg = 0
    sgns = [ever[:-3] for ever in list(map(str, eq.expand().args))]
    for ever in sgns:
        if ever == '-':
            neg += 1
        else:
            pos += 1
    return pos * neg != 0
from collections import defaultdict
from tqdm import tqdm

points = []
edges = []
faces = []
cells = []
strangers = []

obj_eqns = defaultdict(list)
dims_respectively = []
atlas = (points, edges, faces, cells, strangers)

corresp_triple = []
for never_reuse in tqdm(range(len(all_triples))):
    lams = all_triples[never_reuse]
    
    eq1 = create_equation_by_matrix(lams[0], lams[1], 0, 1)
    eq2 = create_equation_by_matrix(lams[0], lams[2], 0, 2)
    eq3 = create_equation_by_matrix(lams[1], lams[2], 1, 2)
    
    norm1 = create_equation_by_matrix(lams[0], lams[0], 0, 0)
    norm2 = create_equation_by_matrix(lams[1], lams[1], 1, 1)
    norm3 = create_equation_by_matrix(lams[2], lams[2], 2, 2)
    
    if not balanced(eq1):
        continue
    if not balanced(eq2):
        continue
    if not balanced(eq3):
        continue

    num_of_eqs = 6

    if eq1 == 0:
        num_of_eqs -= 1
    if eq2 == 0:
        num_of_eqs -= 1
    if eq3 == 0:
        num_of_eqs -= 1
    if norm1 == 0:
        continue
        num_of_eqs -= 1
    if norm2 == 0:
        continue
        num_of_eqs -= 1
    if norm3 == 0:
        continue
        num_of_eqs -= 1
    
    num_of_parameters = len(lams[0]) + len(lams[1]) + len(lams[2])
    obj_eqns[lams] = (eq1, eq2, eq3, norm1 + 1, norm2 + 1, norm3 + 1)
    atlas[num_of_parameters - num_of_eqs].append(lams)
    corresp_triple.append(lams)
    dims_respectively.append(num_of_parameters - num_of_eqs)
def chek_for_consistency(equations, dim):
  variables = set().union(*[eq.free_symbols for eq in equations])
  variables_list = list(variables)
  J_M = []
  for equation in equations:
      grad = Matrix([diff(equation, var) for var in variables_list]).T
      J_M.append(grad)
  Jacobian = Matrix(J_M)
  return (len(variables) - Jacobian.rank()) == dim

1) 1 4 9
2) 1 4 10
3) 1 5 8
4) 1 5 10
5) 1 8 9
6) 2 3 9
7) 2 3 12
8) 2 6 8
9) 2 6 12
10) 2 8 9
11) 3 4 7
12) 3 4 9
13) 3 7 12
14) 4 7 10
15) 5 8 13
16) 5 10 13
17) 6 8 13
18) 6 12 13
19) 7 10 11
20) 7 11 12
21) 10 11 13
22) 11 12 13


100%|██████████| 50116/50116 [02:42<00:00, 308.43it/s]


In [6]:
from collections import Counter
print(Counter(dims_respectively))
point_eqns = {obj_eqns[ever] for ever in atlas[0]}
edge_eqns = {obj_eqns[ever] for ever in atlas[1]}
face_eqns = {obj_eqns[ever] for ever in atlas[2]}
cell_eqns = {obj_eqns[ever] for ever in atlas[3]}


Counter({2: 264, 3: 148, 1: 138, 0: 22})


In [7]:
for obj, system in obj_eqns.items():
    if system == (-c*q + d*(a + 2*b) + f*(a + b),
                    a*h - c*g + k*(a + b),
                    2*d*h - g*q,
                    -a**2 - b**2 - c**2 + 1,
                    -d**2 - q**2 + 1,
                    -g**2 - h**2 + 1):
        print(obj)

((4, 7, 10), (5, 10, 13), (10, 11, 13))


# Получим все включения

In [8]:
points = tuple(points)
edges = tuple(edges)
faces = tuple(faces)
cells = tuple(cells)
flatten_atlas = []
for ever in atlas:
    flatten_atlas += ever
rename_everything = {ob : i for i, ob in enumerate(flatten_atlas)}
setatlas = [[], [], [], []]
for i, ever in enumerate(atlas):
    for lamtype in ever:
        setlamtype = []
        for lamin in lamtype:
            setlamtype.append(set(lamin))
        setatlas[i].append(setlamtype)
point_edge = defaultdict(list)
for i, point in enumerate(setatlas[0]):
   for j, edge in enumerate(setatlas[1]):
      for perm in permutations([0, 1, 2]):
         if point[0].issubset(edge[perm[0]]) and point[1].issubset(edge[perm[1]]) and point[2].issubset(edge[perm[2]]):
            point_edge[atlas[0][i]].append(atlas[1][j])
            break
edge_face = defaultdict(list)
for i, egde in enumerate(setatlas[1]):
   for j, face in enumerate(setatlas[2]):
      for perm in permutations([0, 1, 2]):
         if egde[0].issubset(face[perm[0]]) and egde[1].issubset(face[perm[1]]) and egde[2].issubset(face[perm[2]]):
            edge_face[atlas[1][i]].append(atlas[2][j])
            break
face_cell = defaultdict(list)
for i, face in enumerate(setatlas[2]):
   for j, cell in enumerate(setatlas[3]):
      for perm in permutations([0, 1, 2]):
         if face[0].issubset(cell[perm[0]]) and face[1].issubset(cell[perm[1]]) and face[2].issubset(cell[perm[2]]):
            face_cell[atlas[2][i]].append(atlas[3][j])
            break

In [9]:
point_cell = defaultdict(set)
point_face = defaultdict(set)

for point in point_edge:
    for edge in point_edge[point]:
        for face in edge_face[edge]:
            point_face[point].add(face)
            for cell in face_cell[face]:
                point_cell[point].add(cell)

edge_point = defaultdict(list)
for point in point_edge:
    for edge in point_edge[point]:
        edge_point[edge].append(point)

face_point = defaultdict(list)
for point in point_face:
    for face in point_face[point]:
        face_point[face].append(point)

cell_point = defaultdict(list)
for point in point_cell:
    for cell in point_cell[point]:
        cell_point[cell].append(point)

cell_face = defaultdict(list)
for face in face_cell:
    for cell in face_cell[face]:
        cell_face[cell].append(face)

face_edge = defaultdict(list)
for edge in edge_face:
    for face in edge_face[edge]:
        face_edge[face].append(edge)

In [10]:
cell_face[((1, 4, 9), (1, 5, 8), (1, 5, 10))][0]

((1, 4), (1, 5, 8), (1, 5, 10))

In [ ]:
face_edge[cell_face[((1, 4, 9), (1, 5, 8), (1, 5, 10))][0]][0]

((1,), (5, 8), (5, 10))

In [ ]:
obj_eqns[((1,), (5, 8), (5, 10))]

(0, 0, -d*g + h*q, 1 - a**2, -d**2 - q**2 + 1, -g**2 - h**2 + 1)

In [11]:
obj_eqns[((1, 4, 9), (1, 5, 8), (1, 5, 10))]

(-a*d + b*f + q*(b + c),
 -a*g + c*k + h*(b + c),
 -d*g + f*k - h*q,
 -a**2 - b**2 - c**2 + 1,
 -d**2 - f**2 - q**2 + 1,
 -g**2 - h**2 - k**2 + 1)

In [12]:
equations = obj_eqns[((1, 4, 9), (1, 5, 8), (1, 5, 10))]
variables = set().union(*[eq.free_symbols for eq in equations])
variables_list = list(variables)
J_M = []
for equation in equations:
    grad = Matrix([diff(equation, var) for var in variables_list]).T
    J_M.append(grad)
Jacobian1 = Matrix(J_M)
Jacobian1

Matrix([
[  -a,    0,     0,   -d,    b,    0,     q, f + q, b + c],
[   0,    c, b + c,   -g,    0,   -a, h + k,     h,     0],
[  -g,    f,    -q,    0,    k,   -d,     0,     0,    -h],
[   0,    0,     0, -2*a,    0,    0,  -2*c,  -2*b,     0],
[-2*d,    0,     0,    0, -2*f,    0,     0,     0,  -2*q],
[   0, -2*k,  -2*h,    0,    0, -2*g,     0,     0,     0]])

In [ ]:
obj_eqns[((1, 4), (1, 5, 8), (1, 5, 10))]

(-a*d + b*f + b*q,
 -a*g + b*h,
 -d*g + f*k - h*q,
 -a**2 - b**2 + 1,
 -d**2 - f**2 - q**2 + 1,
 -g**2 - h**2 - k**2 + 1)

In [13]:
equations = obj_eqns[((1, 4), (1, 5, 8), (1, 5, 10))]
J_M = []
for equation in equations:
    grad = Matrix([diff(equation, var) for var in variables_list]).T
    J_M.append(grad)
Jacobian2 = Matrix(J_M)

In [ ]:
variables_list

[q, b, c, a, h, d, f, g, k]

## Проверим клетки

In [ ]:

for system in tqdm(cell_eqns):
    if not chek_for_consistency(system, 3):
        print(system)

100%|██████████| 120/120 [34:33<00:00, 17.28s/it]  


# Проверим faces

In [ ]:
for system in tqdm(face_eqns):
    if not chek_for_consistency(system, 2):
        print(system)

100%|██████████| 171/171 [03:10<00:00,  1.11s/it]


# Проверим ребра

In [ ]:
for system in tqdm(edge_eqns):
    if not chek_for_consistency(system, 1):
        print(system)

100%|██████████| 47/47 [00:23<00:00,  1.96it/s]


# Проверим точки

In [ ]:
for system in tqdm(point_eqns):
    if not chek_for_consistency(system, 0):
        print(system)

100%|██████████| 5/5 [00:05<00:00,  1.03s/it]


# Проверим единственность точек

In [ ]:
for equations in point_eqns:
    variables = set().union(*[eq.free_symbols for eq in equations])
    variables_list = list(variables)
    solutions = solve(equations, variables_list)
    positive_solutions = []
    for solution in solutions:
        if all(coord >= 0 for coord in solution):
            positive_solutions.append(solution)
    print(positive_solutions)
    if len(positive_solutions) != 1:
        print(equations)
        

[(sqrt(6)/3, sqrt(3)/3, sqrt(6)/3, sqrt(6)/3, sqrt(3)/3, sqrt(3)/3)]
[(sqrt(6)/3, sqrt(6)/3, sqrt(3)/3, sqrt(6)/3, sqrt(3)/3, sqrt(3)/3)]
[(sqrt(6)/3, sqrt(3)/3, sqrt(6)/3, sqrt(3)/3, sqrt(3)/3, sqrt(6)/3)]
[(1, 1, 1)]
[(sqrt(2)/2, sqrt(2)/2, sqrt(2)/2, 1, sqrt(2)/2, 1)]


# Проверим единственность ребер

In [ ]:
eqn_solutions = []
for eqns in tqdm(edge_eqns):
    variables = set().union(*[eq.free_symbols for eq in eqns])
    variables_list = list(variables)
    solutions = solve(eqns, variables_list, positive_solutions = True)
    eqn_solutions.append(solutions)

In [ ]:
w = -1

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

0


Matrix([
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, -q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q,  q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), -sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, -q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), -sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q,  q*sqrt(-1/(3*q**2 - 4))*sqrt((

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

1


Matrix([
[-q*sqrt(-1/(3*q**2 - 4)), q, -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, -q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-q*sqrt(-1/(3*q**2 - 4)), q, -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q,  q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-q*sqrt(-1/(3*q**2 - 4)), q,  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, -q*sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)/sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[-q*sqrt(-1/(3*q**2 - 4)), q,  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q,  q*sqrt(-1/(3*q**2 - 4))*sqrt((

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

2


Matrix([
[-2*sqrt((q**2 - 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2),  q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q**2 - 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*(3*q**2 - 2)/q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4), -q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q**2 - 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*(2 - 3*q**2)/q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2), -q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[-2*sqrt((q**2 - 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*(2 - 3*q**2)/q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4),  q, sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[ 2*sqrt((q**2 - 1)/(3*q**2 - 4)), 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

3


Matrix([
[-2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q,  2*sqrt(1/(q**4 + 2*q**2 + 5)),  (q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q,  2*sqrt(1/(q**4 + 2*q**2 + 5)),  (q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ 2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ 2*q*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

4


Matrix([
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)),   2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)), sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8)), -q,  sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)),   2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)),  sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8)),  q,  sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), -2*q*sqrt(-(q**2 - 1)/(9*q**4 - 16*q**2 + 8)), sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8)),  q, sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

5


Matrix([
[-1, -sqrt(1 - q**2), -q, q,  sqrt(-(q - 1)*(q + 1))],
[-1, -sqrt(1 - q**2),  q, q, -sqrt(-(q - 1)*(q + 1))],
[-1,  sqrt(1 - q**2), -q, q, -sqrt(-(q - 1)*(q + 1))],
[-1,  sqrt(1 - q**2),  q, q,  sqrt(-(q - 1)*(q + 1))],
[ 1, -sqrt(1 - q**2), -q, q,  sqrt(-(q - 1)*(q + 1))],
[ 1, -sqrt(1 - q**2),  q, q, -sqrt(-(q - 1)*(q + 1))],
[ 1,  sqrt(1 - q**2), -q, q, -sqrt(-(q - 1)*(q + 1))],
[ 1,  sqrt(1 - q**2),  q, q,  sqrt(-(q - 1)*(q + 1))]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

6


Matrix([
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1)), -q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

7


Matrix([
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), -sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), -sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

8


Matrix([
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

9


Matrix([
[-(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[-(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)),  q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)),  q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[ (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[ (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

10


Matrix([
[-2*q*sqrt(1/(3*q**2 + 1)), q, (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (3*q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (-3*q**4 + 2*q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1)), -q,  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (3*q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  q,  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (1 - 3*q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (-3*q**4 + 2*q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  q, (-3*q**4 + 2*q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (1 - 3*q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

11


Matrix([
[-2*q*sqrt(-(q - 1)*(q + 1)/(21*q**4 - 40*q**2 + 20)), q, (2 - 3*q**2)*sqrt(1/(21*q**4 - 40*q**2 + 20)),  4*(q - 1)*(q + 1)*sqrt(1/(21*q**4 - 40*q**2 + 20)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*sqrt((q**2 - 1)/(3*q**2 - 4))*(21*q**4 - 40*q**2 + 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))/(q**2 - 1),  sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*(21*q**4 - 40*q**2 + 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))],
[-2*q*sqrt(-(q - 1)*(q + 1)/(21*q**4 - 40*q**2 + 20)), q, (2 - 3*q**2)*sqrt(1/(21*q**4 - 40*q**2 + 20)),  4*(q - 1)*(q + 1)*sqrt(1/(21*q**4 - 40*q**2 + 20)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-21*q**4 + 40*q**2 - 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))/(q**2 - 1),  sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*(21*q**4 - 40*q**2 + 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))],
[-2*q*sqrt(-(q - 1)*(q + 1)/(21*q**4 - 40*q**2 + 20)), q, (3*q**2 - 2)*sqrt(1/(21*q**4 - 40*q**2 + 20

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

12


Matrix([
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)),  4*sqrt(-1/(3*q**2 - 4))*(q - 1)*(q + 1)/q, -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), -q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q, -q*sqrt(-1/(3*q**2 - 4)),  4*sqrt(-1/(3*q**2 - 4))*(q - 1)*(q + 1)/q,  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4 - 3*q**2)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), -4*sqrt(-1/(3*q**2 - 4))*(q - 1)*(q + 1)/q, -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[-2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q,  q*sqrt(-1/(3*q**2 - 4)), -4*sqrt(-1/(3*q**2 - 4))*(q - 1)*(q + 1)/q,  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), -q*sqrt(-1/(3*q**2 - 4)), sqrt(-1/(3*q**2 - 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(3*q**2 - 4)],
[ 2*sqrt((q

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

13


Matrix([
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1)),  q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q**2*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

14


Matrix([
[-1, -sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q,  q],
[-1, -sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q, -q],
[-1,  sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q, -q],
[-1,  sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q,  q],
[ 1, -sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q,  q],
[ 1, -sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q, -q],
[ 1,  sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q, -q],
[ 1,  sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q,  q]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

15


Matrix([
[-q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)),  q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)),  q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

16


Matrix([
[(1 - 3*q**2)*sqrt(1/(9*q**4 - 10*q**2 + 5)), q, 2*(q**2 - 1)*sqrt(1/(9*q**4 - 10*q**2 + 5)),   2*q*sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5)), -2*q*sqrt(1/(3*q**2 + 1)),  sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5))*(9*q**4 - 10*q**2 + 5)*sqrt(1/(3*q**2 + 1))*sqrt(1/(9*q**4 - 10*q**2 + 5)), sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5))*(-9*q**4 + 10*q**2 - 5)*sqrt(1/(9*q**4 - 10*q**2 + 5))],
[(1 - 3*q**2)*sqrt(1/(9*q**4 - 10*q**2 + 5)), q, 2*(q**2 - 1)*sqrt(1/(9*q**4 - 10*q**2 + 5)),   2*q*sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5)),  2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5))*(-9*q**4 + 10*q**2 - 5)*sqrt(1/(3*q**2 + 1))*sqrt(1/(9*q**4 - 10*q**2 + 5)), sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5))*(-9*q**4 + 10*q**2 - 5)*sqrt(1/(9*q**4 - 10*q**2 + 5))],
[(1 - 3*q**2)*sqrt(1/(9*q**4 - 10*q**2 + 5)), q, 2*(q**2 - 1)*sqrt(1/(9*q**4 - 10*q**2 + 5)), -2*q*sqrt(-(q**2 - 1)/(9*q**4 - 10*q**2 + 5)), -2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(9*q**4 - 10*q**2 + 5))*(-9*q**4 + 10*q**2 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

17


Matrix([
[-1, sqrt(-1/(q**2 - 1))*(1 - q**2),        -q*sqrt(-1/(q**2 - 1)), q, -1],
[-1, sqrt(-1/(q**2 - 1))*(1 - q**2),         q*sqrt(-1/(q**2 - 1)), q,  1],
[-1, sqrt(-1/(q**2 - 1))*(q**2 - 1), -q*sqrt(-1/((q - 1)*(q + 1))), q,  1],
[-1, sqrt(-1/(q**2 - 1))*(q**2 - 1),  q*sqrt(-1/((q - 1)*(q + 1))), q, -1],
[ 1, sqrt(-1/(q**2 - 1))*(1 - q**2),        -q*sqrt(-1/(q**2 - 1)), q, -1],
[ 1, sqrt(-1/(q**2 - 1))*(1 - q**2),         q*sqrt(-1/(q**2 - 1)), q,  1],
[ 1, sqrt(-1/(q**2 - 1))*(q**2 - 1), -q*sqrt(-1/((q - 1)*(q + 1))), q,  1],
[ 1, sqrt(-1/(q**2 - 1))*(q**2 - 1),  q*sqrt(-1/((q - 1)*(q + 1))), q, -1]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

18


Matrix([
[-1, -sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q,  q],
[-1, -sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q, -q],
[-1,  sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q, -q],
[-1,  sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q,  q],
[ 1, -sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q,  q],
[ 1, -sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q, -q],
[ 1,  sqrt(1 - q**2), -sqrt(-(q - 1)*(q + 1)), q, -q],
[ 1,  sqrt(1 - q**2),  sqrt(-(q - 1)*(q + 1)), q,  q]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

19


Matrix([
[sqrt(-1/(q**2 - 1))*(1 - q**2), q, -q,  q**2*sqrt(-1/((q - 1)*(q + 1))), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(q**2 - 1)],
[sqrt(-1/(q**2 - 1))*(1 - q**2), q, -q,  q**2*sqrt(-1/((q - 1)*(q + 1))),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(q**2 - 1)],
[sqrt(-1/(q**2 - 1))*(1 - q**2), q,  q,         q**2*sqrt(-1/(q**2 - 1)),      -2*sqrt((q**2 - 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 - q**2)],
[sqrt(-1/(q**2 - 1))*(1 - q**2), q,  q,         q**2*sqrt(-1/(q**2 - 1)),       2*sqrt((q**2 - 1)/(3*q**2 - 4)),  q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 - q**2)],
[sqrt(-1/(q**2 - 1))*(q**2 - 1), q, -q,        -q**2*sqrt(-1/(q**2 - 1)),      -2*sqrt((q**2 - 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 -

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

20


Matrix([
[-q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(2*q**4 - q**2 + 1)),  q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(2*q**4 - q**2 + 1)),  q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/(2*q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(2*q**4 - q**2 + 1)), -q*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/(2*q**4 - q**2 + 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

21


Matrix([
[-2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)), q, (2 - 3*q**2)*sqrt(1/(9*q**4 - 8*q**2 + 4)),  2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))],
[-2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)), q, (2 - 3*q**2)*sqrt(1/(9*q**4 - 8*q**2 + 4)),  2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(9*q**4 - 8*q**2 + 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))],
[-2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 8*q**2 + 4)), -2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

22


Matrix([
[-q, q, sqrt(-1/(q**2 - 1))*(1 - q**2),  q**2*sqrt(-1/((q - 1)*(q + 1))), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(q**2 - 1)],
[-q, q, sqrt(-1/(q**2 - 1))*(1 - q**2),  q**2*sqrt(-1/((q - 1)*(q + 1))),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(q**2 - 1)],
[-q, q, sqrt(-1/(q**2 - 1))*(q**2 - 1),        -q**2*sqrt(-1/(q**2 - 1)),      -2*sqrt((q**2 - 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 - q**2)],
[-q, q, sqrt(-1/(q**2 - 1))*(q**2 - 1),        -q**2*sqrt(-1/(q**2 - 1)),       2*sqrt((q**2 - 1)/(3*q**2 - 4)),  q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 - q**2)],
[ q, q, sqrt(-1/(q**2 - 1))*(1 - q**2),         q**2*sqrt(-1/(q**2 - 1)),      -2*sqrt((q**2 - 1)/(3*q**2 - 4)), -q*sqrt(-1/(q**2 - 1))*sqrt((q**2 - 1)/(3*q**2 - 4)), sqrt(-1/(q**2 - 1))*(1 -

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

23


Matrix([
[-2*q*sqrt(1/(3*q**2 + 1)), q, (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  4*q**2*sqrt(1/(-3*q**4 + 2*q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)), (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  4*q**2*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  2*q*sqrt(1/(3*q**2 + 1)), (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  (3*q**4 - 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), -4*q**2*sqrt(1/(-3*q**4 + 2*q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)), (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (-3*q**4 + 2*q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(-3*q**4 + 2*q**2 + 1))],
[-2*q*sqrt(1/(3*q**2 + 1)), q, (q**2 - 1)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), -4*q**2*sqrt(1/(-3*q**4 + 2*q**2 + 1)),  2*q*sqrt(1/(3*q**2 + 1)), (1 - q**2)*sqrt(1/(-3*q**4 + 2*q**2 + 1)), (-3*q**4 + 2*q**2 + 1)*s

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

24


Matrix([
[-(3*q**2 - 2)*sqrt(1/(9*q**4 - 8*q**2 + 4)), q,  2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)), -2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))],
[-(3*q**2 - 2)*sqrt(1/(9*q**4 - 8*q**2 + 4)), q,  2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)), -2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(9*q**4 - 8*q**2 + 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(9*q**4 - 8*q**2 + 4))*(-9*q**4 + 8*q**2 - 4)*sqrt(1/(9*q**4 - 8*q**2 + 4))],
[-(3*q**2 - 2)*sqrt(1/(9*q**4 - 8*q**2 + 4)), q,  2*q**2*sqrt(1/(9*q**4 - 8*q**2 + 4)),  2*q*sqrt(-(q - 1)*(q + 1)/(9*q**4 - 8*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

25


Matrix([
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)),  q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)),  q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

26


Matrix([
[-2*(q - 1)*(q + 1)*sqrt(1/(4*q**4 - 7*q**2 + 4)), q,  q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)), -q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))],
[-2*(q - 1)*(q + 1)*sqrt(1/(4*q**4 - 7*q**2 + 4)), q,  q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)), -q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4*q**4 - 7*q**2 + 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))],
[-2*(q - 1)*(q + 1)*sqrt(1/(4*q**4 - 7*q**2 + 4)), q,  q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)),  q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 -

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

27


Matrix([
[  2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)), q, (2 - 3*q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), 2*(q**2 - 1)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2),  sqrt(1 - q**2), sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[  2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)), q, (2 - 3*q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), 2*(q**2 - 1)*sqrt(1/(9*q**4 - 16*q**2 + 8)),  q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2), -sqrt(1 - q**2), sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[  2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), 2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2), -

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

28


Matrix([
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, -q**2*sqrt(1/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, -q**2*sqrt(1/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1)), -q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)), (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)), (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

29


Matrix([
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*q**2, -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*q**2,  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*q**2, -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*q**2,  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*q**2, -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*q**2,  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*q**2, -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*q**2,  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

30


Matrix([
[-4*q**2*sqrt(1/(21*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(21*q**4 - 2*q**2 + 1)),   2*q*sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)),  sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1))*(21*q**4 - 2*q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(21*q**4 - 2*q**2 + 1)), sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1))*(-21*q**4 + 2*q**2 - 1)*sqrt(1/(21*q**4 - 2*q**2 + 1))],
[-4*q**2*sqrt(1/(21*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(21*q**4 - 2*q**2 + 1)),   2*q*sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1)),  2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1))*(-21*q**4 + 2*q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(21*q**4 - 2*q**2 + 1)), sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1))*(-21*q**4 + 2*q**2 - 1)*sqrt(1/(21*q**4 - 2*q**2 + 1))],
[-4*q**2*sqrt(1/(21*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(21*q**4 - 2*q**2 + 1)), -2*q*sqrt(-(q**2 - 1)/(21*q**4 - 2*q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(21*q**4 - 2*q**2 + 1))*(-21*q**4 + 2*q**2 - 1)*sqrt(1/(3*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

31


Matrix([
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1)),  q,  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  q, sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt((1 - q**2)/(q**4 - q**2 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

32


Matrix([
[-4*(q - 1)*(q + 1)*sqrt(1/(21*q**4 - 40*q**2 + 20)), q, (3*q**2 - 2)*sqrt(1/(21*q**4 - 40*q**2 + 20)), -2*q*sqrt(-(q - 1)*(q + 1)/(21*q**4 - 40*q**2 + 20)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-21*q**4 + 40*q**2 - 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))/(q**2 - 1), sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*(-21*q**4 + 40*q**2 - 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))],
[-4*(q - 1)*(q + 1)*sqrt(1/(21*q**4 - 40*q**2 + 20)), q, (3*q**2 - 2)*sqrt(1/(21*q**4 - 40*q**2 + 20)), -2*q*sqrt(-(q - 1)*(q + 1)/(21*q**4 - 40*q**2 + 20)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*sqrt((q**2 - 1)/(3*q**2 - 4))*(21*q**4 - 40*q**2 + 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))/(q**2 - 1), sqrt((1 - q**2)/(21*q**4 - 40*q**2 + 20))*(-21*q**4 + 40*q**2 - 20)*sqrt(1/(21*q**4 - 40*q**2 + 20))],
[-4*(q - 1)*(q + 1)*sqrt(1/(21*q**4 - 40*q**2 + 20)), q, (3*q**2 - 2)*sqrt(1/(21*q**4 - 40*q**2 + 20)

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

33


Matrix([
[-(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -2*q*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)), -2*q*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)),  2*q*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-(q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q, -2*sqrt(1/(q**4 + 2*q**2 + 5)),  2*q*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ (q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q,  2*sqrt(1/(q**4 + 2*q**2 + 5)), -2*q*sqrt(1/(q**4 + 2*q**2 + 5)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ (q - 1)*(q + 1)*sqrt(1/(q**4 + 2*q**2 + 5)), q,  2*sqrt(1/(q**4 + 2*q**2 + 5)), -2*q*sqrt(1/(q**4 + 2*q**2 + 5)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

34


Matrix([
[1 - 2*q**2, q, -2*q*sqrt(-(q - 1)*(q + 1)),  2*q**2, -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[1 - 2*q**2, q, -2*q*sqrt(-(q - 1)*(q + 1)),  2*q**2,  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[1 - 2*q**2, q,  2*q*sqrt(-(q - 1)*(q + 1)),  2*q**2, -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[1 - 2*q**2, q,  2*q*sqrt(-(q - 1)*(q + 1)),  2*q**2,  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[2*q**2 - 1, q, -2*q*sqrt(-(q - 1)*(q + 1)), -2*q**2, -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[2*q**2 - 1, q, -2*q*sqrt(-(q - 1)*(q + 1)), -2*q**2,  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[2*q**2 - 1, q,  2*q*sqrt(-(q - 1)*(q + 1)), -2*q**2, -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[2*q**2 - 1, q,  2*q*sqrt(-(q - 1)*(q + 1)), -2*q**2,  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

35


Matrix([
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)),   2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)), q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2), -sqrt(1 - q**2),  sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)),   2*q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8)),  q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2),  sqrt(1 - q**2),  sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(9*q**4 - 16*q**2 + 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))],
[2*(1 - q**2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), q, (3*q**2 - 2)*sqrt(1/(9*q**4 - 16*q**2 + 8)), -2*q*sqrt(-(q**2 - 1)/(9*q**4 - 16*q**2 + 8)), q*sqrt((1 - q**2)/(9*q**4 - 16*q**2 + 8))*(-9*q**4 + 16*q**2 - 8)*sqrt(1/(9*q**4 - 16*q**2 + 8))/sqrt(1 - q**2),  

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

36


Matrix([
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, -q**2*sqrt(1/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, -q**2*sqrt(1/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)), (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q,  q**2*sqrt(

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

37


Matrix([
[-2*q**2*sqrt(1/(4*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(4*q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(4*q**4 - q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)),  sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(4*q**4 - q**2 + 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(4*q**4 - q**2 + 1)), sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(-4*q**4 + q**2 - 1)*sqrt(1/(4*q**4 - q**2 + 1))],
[-2*q**2*sqrt(1/(4*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(4*q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(4*q**4 - q**2 + 1)),  2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(-4*q**4 + q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(4*q**4 - q**2 + 1)), sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(-4*q**4 + q**2 - 1)*sqrt(1/(4*q**4 - q**2 + 1))],
[-2*q**2*sqrt(1/(4*q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(4*q**4 - q**2 + 1)), -q*sqrt(-(q**2 - 1)/(4*q**4 - q**2 + 1)), -2*q*sqrt(1/(3*q**2 + 1)), sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(-4*q**4 + q**2 - 1)*sqrt(1/(3*q**2 + 1))*sqrt(1/(4*q**4 - q**2 + 1)),  sqrt((1 - q**2)/(4*q**4 - q**2 + 1))*(4*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

38


Matrix([
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)),  q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q, -sqrt(1/(q**4 + q**2 + 1)),  q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ q**2*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))), q,  sqrt(1/(q**4 + q**2 + 1)), -q*sqrt(1/((q**2 - q + 1)*(q**2 + q + 1))),  sqrt(1/

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

39


Matrix([
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*(q - 1)*(q + 1), -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*(q - 1)*(q + 1),  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*(q - 1)*(q + 1), -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[-2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*(q - 1)*(q + 1),  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*(q - 1)*(q + 1), -q,  sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 1 - 2*q**2,  2*(q - 1)*(q + 1),  q, -sqrt(-(q - 1)*(q + 1)), -sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*(q - 1)*(q + 1), -q, -sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)],
[ 2*q*sqrt(-(q - 1)*(q + 1)), q, 2*q**2 - 1, -2*(q - 1)*(q + 1),  q,  sqrt(-(q - 1)*(q + 1)),  sqrt(1 - q**2)]])

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

40


Matrix([
[-q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)), q, 2*(1 - q**2)*sqrt(1/(4*q**4 - 7*q**2 + 4)),  q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)), q*sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))],
[-q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)), q, 2*(1 - q**2)*sqrt(1/(4*q**4 - 7*q**2 + 4)),  q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)),  2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*sqrt((q**2 - 1)/(3*q**2 - 4))*(4*q**4 - 7*q**2 + 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))/(q**2 - 1), sqrt((1 - q**2)/(4*q**4 - 7*q**2 + 4))*(-4*q**4 + 7*q**2 - 4)*sqrt(1/(4*q**4 - 7*q**2 + 4))],
[-q*sqrt(-(q - 1)*(q + 1)/(4*q**4 - 7*q**2 + 4)), q, 2*(q**2 - 1)*sqrt(1/(4*q**4 - 7*q**2 + 4)), -q**2*sqrt(1/(4*q**4 - 7*q**2 + 4)), -2*sqrt((q - 1)*(q + 1)/(3*q**2 - 4)),  q*sqrt((1 -

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

41


Matrix([
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), -sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

42


Matrix([
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (1 - q**2)*sqrt(1/(q**4 - q**2 + 1)),  q**2*sqrt(1/(q**4 - q**2 + 1)),  sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -q**2*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[  q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), q, (q**2 - 1)*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

43


Matrix([
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q,   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)),  sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)), (q**2 - 1)*sqrt(1/(q**4 - q**2 + 1)), -sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[-q**2*sqrt(1/(q**4 - q**2 + 1)), q, -q*sqrt(-(q**2 - 1

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

44


Matrix([
[-2*q**2*sqrt(1/(9*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)),   2*q*sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1)), sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(-9*q**4 + 2*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)),  q, sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(-9*q**4 + 2*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1))],
[-2*q**2*sqrt(1/(9*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)),   2*q*sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1)),  sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(9*q**4 - 2*q**2 + 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)), -q, sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(-9*q**4 + 2*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1))],
[-2*q**2*sqrt(1/(9*q**4 - 2*q**2 + 1)), q, (3*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)), -2*q*sqrt(-(q**2 - 1)/(9*q**4 - 2*q**2 + 1)), sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(-9*q**4 + 2*q**2 - 1)*sqrt(1/(9*q**4 - 2*q**2 + 1)), -q,  sqrt((1 - q**2)/(9*q**4 - 2*q**2 + 1))*(9*q**4 - 2*q**2 + 1)*sqrt(1/(9*q**4 - 2*q**2 + 1))],
[-2*q**2*sqrt(1/(9*

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

45


Matrix([
[-q*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)),  (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)),  1],
[-q*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)),  (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)),  1],
[-q*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)), -(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[-q*sqrt(1/(2*q**4 - q**2 + 1)), q,  q**2*sqrt(1/(2*q**4 - q**2 + 1)), -(q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), -1],
[ q*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)),  (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)), -sqrt(1/(q**2 + 1)),  q*sqrt(1/(q**2 + 1)), -1],
[ q*sqrt(1/(2*q**4 - q**2 + 1)), q, -q**2*sqrt(1/(2*q**4 - q**2 + 1)),  (q - 1)*(q + 1)*sqrt(1/(2*q**4 - q**2 + 1)),  sqrt(1/(q**2 + 1)), -q*sqrt(1/(q**2 + 1)), 

In [ ]:
w += 1
print(w)
Matrix(eqn_solutions[w])

46


Matrix([
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)), -sqrt(1 - q**2), q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)),   q*sqrt((1 - q**2)/(q**4 - q**2 + 1)),  sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2),  sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/(q**4 - q**2 + 1)), -q*sqrt(-(q**2 - 1)/(q**4 - q**2 + 1)), -sqrt(1 - q**2),  q*sqrt((1 - q**2)/(q**4 - q**2 + 1))*(q**4 - q**2 + 1)*sqrt(1/(q**4 - q**2 + 1))/sqrt(1 - q**2), sqrt((1 - q**2)/(q**4 - q**2 + 1))*(-q**4 + q**2 - 1)*sqrt(1/(q**4 - q**2 + 1))],
[(1 - q**2)*sqrt(1/(q**4 - q**2 + 1)), q,  q**2*sqrt(1/

In [14]:
cell_eqns = list(cell_eqns)

In [ ]:
def reverse(equation):
    renewed = ''
    if equation == 0:
        return 0
    for ch in equation:
        if ch == "+":
            renewed += '-'
        elif ch == "-":
            renewed += '+'
        else:
            renewed += ch
    return renewed

def generate_synonims(eqns):
    eq1 = eqns[0]
    eq2 = eqns[1]
    eq3 = eqns[2]
    letters1 = ['a', 'b', 'c']
    letters2 = ['d', 'q', 'f']
    letters3 = ['g', 'h', 'k']
    synonims = set()
    for perm1 in permutations([0, 1, 2]):
        for perm2 in permutations([0, 1, 2]):
            for perm3 in permutations([0, 1, 2]):
                grand_subs = {'a' : letters1[perm1[0]], 'b' : letters1[perm1[1]], 'c' : letters1[perm1[2]],
                                'd' : letters2[perm2[0]], 'q' : letters2[perm2[1]], 'f' : letters2[perm2[2]],
                                'g' : letters3[perm3[0]], 'h' : letters3[perm3[1]], 'k' : letters3[perm3[2]]}
                if eq1 != 0:
                    eq1 = str(eq1)
                    replaced_chars = [grand_subs.get(char, char) for char in eq1]
                    eq1 = ''.join(replaced_chars)

                if eq2 != 0:
                    eq2 = str(eq2)
                    replaced_chars = [grand_subs.get(char, char) for char in eq2]
                    eq2 = ''.join(replaced_chars)

                if eq3 != 0:
                    eq3 = str(eq3)
                    replaced_chars = [grand_subs.get(char, char) for char in eq3]
                    eq3 = ''.join(replaced_chars)

                synonims.add((eq1, eq2, eq3))
                synonims.add((eq1, reverse(eq2), eq3))
                synonims.add((eq1, eq2, reverse(eq3)))
                synonims.add((eq1, reverse(eq2), reverse(eq3)))
                synonims.add((reverse(eq1), eq2, reverse(eq3)))
                synonims.add((reverse(eq1), reverse(eq2), eq3))
                synonims.add((reverse(eq1), eq2, eq3))
                synonims.add((reverse(eq1), reverse(eq2), reverse(eq3)))
    return synonims
str_eqns = set()
for ever in set(cell_eqns):
    eq1 = ever[0].expand()
    eq2 = ever[1].expand()
    eq3 = ever[2].expand()
    if eq1 != 0:
        eq1 = str(eq1)
        if not (eq1[0] in {'+', '-'}):
            eq1 = '+' + eq1
    if eq2 != 0:
        eq2 = str(eq2)
        if not (eq2[0] in {'+', '-'}):
            eq2 = '+' + eq2
    if eq3 != 0:
        eq3 = str(eq3)
        if not (eq3[0] in {'+', '-'}):
            eq3 = '+' + eq3
    str_eqns.add((eq1, eq2, eq3))

In [ ]:
eqns_classes = set()
while str_eqns:
    eqns = str_eqns.pop()
    str_eqns -= generate_synonims(eqns)
    eqns_classes.add(eqns)

In [17]:
for eqn in cell_eqns[0][:3]:
    print(eqn)

b*d - c*f + q*(a + 2*b)
-c*h + g*(a + 2*b) + k*(2*a + 2*b)
d*k - f*h - g*q


a, d, g;

In [18]:
for eqn in cell_eqns[1][:3]:
    print(eqn)

-a*d + b*q - c*f
-c*h + g*(2*a + b) + k*(a + b)
d*k - f*h + g*(2*d + 2*q)


In [19]:
for eqn in cell_eqns[2][:3]:
    print(eqn)

-a*d + b*f + q*(b + c)
-a*g + b*h - c*k
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[3][:3]:
    print(eqn)

-c*q + d*(a + 2*b) + f*(a + b)
a*g - c*h + k*(a + b)
2*d*g - h*q


In [ ]:
for eqn in cell_eqns[4][:3]:
    print(eqn)

b*q - c*f + d*(a + b)
a*g - b*h - c*k
-f*k + g*q + h*(d + q)


In [ ]:
for eqn in cell_eqns[5][:3]:
    print(eqn)

-a*d - b*q + c*f
-a*g + b*h - c*k
-d*g + f*k + h*(f + q)


In [ ]:
for eqn in cell_eqns[6][:3]:
    print(eqn)

a*q - b*d - c*f
-c*h + g*(2*a + b) + k*(a + b)
d*g - f*h + k*(d + q)


In [ ]:
for eqn in cell_eqns[7][:3]:
    print(eqn)

a*d - c*f + q*(a + b)
a*g - b*h - c*k
-d*g - f*k + h*q


In [ ]:
for eqn in cell_eqns[8][:3]:
    print(eqn)

a*f - b*d - c*q
a*h - c*g + k*(a + b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[9][:3]:
    print(eqn)

-c*q + d*(a + 2*b) + f*(a + b)
a*h - b*g - c*k
2*d*h + g*(2*d + f) - k*q


In [ ]:
for eqn in cell_eqns[10][:3]:
    print(eqn)

-a*d + b*q + f*(b + c)
-a*g + c*k + h*(b + c)
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[11][:3]:
    print(eqn)

a*f - b*d - c*q
a*k + b*h - c*g
d*h - g*q


In [ ]:
for eqn in cell_eqns[12][:3]:
    print(eqn)

2*a*q - b*f + d*(2*a + c)
2*a*g - b*h
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[13][:3]:
    print(eqn)

a*d - b*q + f*(a + c)
a*g - b*h - c*k
-d*g + f*k - h*q


In [ ]:
for eqn in cell_eqns[14][:3]:
    print(eqn)

-a*d + b*f + q*(b + c)
-a*g + b*h - c*k
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[15][:3]:
    print(eqn)

a*d - b*f + q*(2*a + c)
a*g - b*h - c*k
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[16][:3]:
    print(eqn)

a*f - b*d + q*(a + c)
a*g - c*k + h*(2*a + b)
-f*g + h*(d + q) + k*q


In [ ]:
for eqn in cell_eqns[17][:3]:
    print(eqn)

a*q - b*d - c*f
a*g - c*k + h*(2*a + b)
d*h - f*k - g*q


In [ ]:
for eqn in cell_eqns[18][:3]:
    print(eqn)

a*f - b*d - c*q
b*k - c*g + h*(a + b)
d*h - g*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[19][:3]:
    print(eqn)

-a*d + b*q - c*f
-c*g + h*(2*a + b) + k*(a + b)
d*k - f*g + h*(2*d + 2*q)


In [ ]:
for eqn in cell_eqns[20][:3]:
    print(eqn)

a*f - b*q + d*(a + c)
a*g - c*h + k*(2*a + b)
d*h - f*g + k*(d + q)


In [ ]:
for eqn in cell_eqns[21][:3]:
    print(eqn)

-a*d + c*f + q*(b + c)
-b*g + c*k + h*(a + c)
-f*k + g*q + h*(d + 2*q)


In [ ]:
for eqn in cell_eqns[22][:3]:
    print(eqn)

a*d - c*q + f*(2*a + b)
-c*g + h*(2*a + b) + k*(a + b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[23][:3]:
    print(eqn)

b*d - c*f + q*(a + 2*b)
a*k - c*h + g*(2*a + 2*b)
d*g - f*h + k*(d + q)


In [ ]:
for eqn in cell_eqns[24][:3]:
    print(eqn)

a*q - b*d - c*f
-b*g + c*k + h*(a + c)
-d*g + f*h + k*(f + q)


In [ ]:
for eqn in cell_eqns[25][:3]:
    print(eqn)

a*q - b*d + f*(a + c)
a*h - b*g - c*k
-d*g + f*k - h*q


In [ ]:
for eqn in cell_eqns[26][:3]:
    print(eqn)

a*d - b*q - c*f
-b*h + g*(2*a + c) + k*(a + c)
f*g - h*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[27][:3]:
    print(eqn)

a*f - b*q + d*(a + c)
-b*h + c*k + g*(a + c)
-d*g + f*k - h*q


In [ ]:
for eqn in cell_eqns[28][:3]:
    print(eqn)

-a*d + c*q + f*(b + c)
a*k - b*g + c*h
f*g - h*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[29][:3]:
    print(eqn)

a*d - b*f + q*(2*a + c)
-b*h + g*(2*a + c) + k*(a + c)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[30][:3]:
    print(eqn)

-a*d + b*q - c*f
a*h - b*g - c*k
-f*k + g*q + h*(d + 2*q)


In [ ]:
for eqn in cell_eqns[31][:3]:
    print(eqn)

a*q - c*f + d*(a + b)
a*g - b*h + k*(a + c)
d*h - g*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[32][:3]:
    print(eqn)

-a*d - b*q + c*f
a*g - b*h + k*(a + c)
d*k + g*(d + f) - h*q


In [ ]:
for eqn in cell_eqns[33][:3]:
    print(eqn)

-a*d + c*f + q*(b + c)
-b*g + c*k + h*(a + c)
-f*k + g*q + h*(d + q)


In [ ]:
for eqn in cell_eqns[34][:3]:
    print(eqn)

-b*q + c*d + f*(a + c)
a*h - b*g - c*k
d*h - g*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[35][:3]:
    print(eqn)

-a*d + b*f - c*q
b*k - c*g + h*(a + b)
d*h - f*k - g*q


In [ ]:
for eqn in cell_eqns[36][:3]:
    print(eqn)

a*q - b*d - c*f
b*g - c*h + k*(a + b)
-f*h + g*(d + 2*q) + k*(d + q)


In [ ]:
for eqn in cell_eqns[37][:3]:
    print(eqn)

-a*d - b*q + c*f
-a*g + c*k + h*(b + c)
-d*g - f*k + h*q


In [ ]:
for eqn in cell_eqns[38][:3]:
    print(eqn)

a*d - b*f + q*(2*a + c)
-b*h + c*g + k*(a + c)
d*k - f*h + g*(2*d + 2*q)


In [ ]:
for eqn in cell_eqns[39][:3]:
    print(eqn)

-b*q + c*d + f*(a + c)
a*k - b*g + h*(a + c)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[40][:3]:
    print(eqn)

-a*d - b*q + c*f
a*h - b*g + k*(a + c)
d*k - g*q + h*(d + f)


In [ ]:
for eqn in cell_eqns[41][:3]:
    print(eqn)

-c*f + d*(a + 2*b) + q*(2*a + 2*b)
b*k - c*h + g*(a + 2*b)
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[42][:3]:
    print(eqn)

-b*q + d*(2*a + c) + f*(a + c)
-b*h + c*g + k*(a + c)
2*d*g - h*q


In [ ]:
for eqn in cell_eqns[43][:3]:
    print(eqn)

b*d - c*f + q*(a + 2*b)
-c*h + g*(a + 2*b) + k*(2*a + 2*b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[44][:3]:
    print(eqn)

a*f - b*d + q*(a + c)
-b*g + c*k + h*(a + c)
-d*g + f*k - h*q


In [ ]:
for eqn in cell_eqns[45][:3]:
    print(eqn)

a*d - b*q - c*f
b*h - c*k + g*(a + b)
-f*k + g*q + h*(d + q)


In [ ]:
for eqn in cell_eqns[46][:3]:
    print(eqn)

a*f - b*d + c*q
a*k - b*g - c*h
-d*g + h*q


In [ ]:
for eqn in cell_eqns[47][:3]:
    print(eqn)

a*q - b*d + f*(a + c)
-b*g + c*k + h*(a + c)
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[48][:3]:
    print(eqn)

a*q - b*d + f*(a + c)
a*h - c*g + k*(a + b)
d*k + f*g - h*q


In [ ]:
for eqn in cell_eqns[49][:3]:
    print(eqn)

-a*d - b*q + c*f
a*k - b*g + c*h
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[50][:3]:
    print(eqn)

-a*d - b*q + c*f
a*g - b*h - c*k
f*k + g*(d + f) - h*q


In [ ]:
for eqn in cell_eqns[51][:3]:
    print(eqn)

-a*d - b*q + c*f
a*k - b*g - c*h
d*k + f*h - g*q


In [ ]:
for eqn in cell_eqns[52][:3]:
    print(eqn)

-a*d + b*f - c*q
-c*k + g*(a + 2*b) + h*(2*a + 2*b)
d*g + h*(2*d + f) - k*q


In [ ]:
for eqn in cell_eqns[53][:3]:
    print(eqn)

a*q - b*d - c*f
a*g - c*h + k*(a + b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[54][:3]:
    print(eqn)

-a*d + b*q - c*f
b*g - c*h + k*(a + b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[55][:3]:
    print(eqn)

-a*d + b*q + f*(b + c)
-a*g + b*h - c*k
-d*g + f*k - h*q


In [ ]:
for eqn in cell_eqns[56][:3]:
    print(eqn)

a*d - c*f + q*(a + b)
b*h - c*k + g*(a + b)
d*h - f*k - g*q


In [ ]:
for eqn in cell_eqns[57][:3]:
    print(eqn)

a*q - c*d + f*(a + b)
b*k - c*g + h*(a + b)
-d*g - f*h + k*q


In [ ]:
for eqn in cell_eqns[58][:3]:
    print(eqn)

a*d - c*f + q*(2*a + b)
-c*h + g*(2*a + b) + k*(a + b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[59][:3]:
    print(eqn)

-a*d + b*q - c*f
-c*h + g*(2*a + b) + k*(a + b)
d*k - f*h + g*(2*d + 2*q)


In [ ]:
for eqn in cell_eqns[60][:3]:
    print(eqn)

-a*d - b*q + c*f
-b*g + c*k + h*(a + c)
d*h - f*k - g*q


In [ ]:
for eqn in cell_eqns[61][:3]:
    print(eqn)

b*d - c*q + f*(a + b)
a*g - c*h + k*(2*a + b)
2*d*k + g*(2*d + f) - h*q


In [ ]:
for eqn in cell_eqns[62][:3]:
    print(eqn)

a*q - b*d - c*f
-c*g + h*(2*a + b) + k*(a + b)
d*h - f*g + k*(d + q)


In [ ]:
for eqn in cell_eqns[64][:3]:
    print(eqn)

-a*d + b*f + q*(b + c)
b*h - c*k + g*(a + b)
-f*h + g*(d + 2*q) + k*q


In [ ]:
for eqn in cell_eqns[65][:3]:
    print(eqn)

-c*f + d*(a + 2*b) + q*(2*a + 2*b)
a*k - c*h + g*(2*a + 2*b)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[66][:3]:
    print(eqn)

a*d - b*q + f*(a + c)
-b*g + c*k + h*(a + c)
d*k - f*h - g*q


In [ ]:
for eqn in cell_eqns[67][:3]:
    print(eqn)

a*d - c*f + q*(a + b)
a*g - b*k + h*(2*a + c)
-d*g + h*(f + q) + k*q


In [ ]:
for eqn in cell_eqns[68][:3]:
    print(eqn)

-a*d + b*q - c*f
b*h - c*k + g*(a + b)
d*g - f*k - h*q


In [ ]:
for eqn in cell_eqns[69][:3]:
    print(eqn)

a*f - b*d + q*(a + c)
a*g - c*h + k*(a + b)
d*k - f*g + h*q


In [ ]:
for eqn in cell_eqns[70][:3]:
    print(eqn)

-a*d + b*f - c*q
a*k - b*g - c*h
f*g - h*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[71][:3]:
    print(eqn)

b*d - c*q + f*(a + b)
-c*g + h*(2*a + b) + k*(a + b)
2*d*h - g*q


In [ ]:
for eqn in cell_eqns[72][:3]:
    print(eqn)

-a*d + c*q + f*(b + c)
-b*g + c*h + k*(a + c)
f*g - h*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[73][:3]:
    print(eqn)

-a*d - b*q + c*f
-a*g + b*k + h*(b + c)
-d*g + h*q + k*(f + q)


In [ ]:
for eqn in cell_eqns[74][:3]:
    print(eqn)

-a*d + c*f + q*(b + c)
-a*g + b*h - c*k
-d*g + f*h + k*(f + q)


In [ ]:
for eqn in cell_eqns[75][:3]:
    print(eqn)

a*f - b*d - c*q
-c*h + g*(a + 2*b) + k*(2*a + 2*b)
2*d*k + g*(2*d + f) - h*q


In [ ]:
for eqn in cell_eqns[76][:3]:
    print(eqn)

b*f - c*q + d*(a + 2*b)
a*k - c*h + g*(2*a + 2*b)
f*g - h*q + k*(d + f)


In [ ]:
for eqn in cell_eqns[77][:3]:
    print(eqn)

-a*d - b*q + c*f
a*h - b*g - c*k
f*k - g*q + h*(d + f)


In [ ]:
for eqn in cell_eqns[78][:3]:
    print(eqn)

-a*d + b*q - c*f
-a*g + c*k + h*(b + c)
-d*g + f*h + k*(f + q)


In [ ]:
for eqn in cell_eqns[79][:3]:
    print(eqn)

a*q - c*f + d*(a + b)
b*h - c*k + g*(a + b)
-d*g - f*k + h*q


In [ ]:
for eqn in cell_eqns[80][:3]:
    print(eqn)

-a*d + b*f - c*q
a*h - c*g + k*(a + b)
d*k - g*q + h*(d + f)
